# Battleship Model Comparison

This notebook uses the refactored Python helpers in `battleship-test/battleship`
to compare multiple Bayesian search strategies on the same boards.


In [ ]:
import sys
from pathlib import Path

import numpy as np

repo_root = Path.cwd()
if (repo_root / "battleship-test" / "battleship").exists():
    sys.path.insert(0, str(repo_root / "battleship-test"))
elif (repo_root / "battleship").exists():
    sys.path.insert(0, str(repo_root))

from battleship.config import BenchmarkConfig, GameConfig, MonteCarloConfig, SearchConfig
from battleship.board import generate_board
from battleship.evaluate import benchmark_algorithms
from battleship.plotting import plot_board
from battleship.priors import center_bias_prior, edge_bias_prior, uniform_prior
from battleship.search import ALGORITHMS


In [ ]:
game_config = GameConfig()
search_config = SearchConfig()
mc_config = MonteCarloConfig()
rng = np.random.default_rng(298)

board = generate_board(rng, game_config)
plot_board(board)


In [ ]:
prior = uniform_prior(game_config.board_size)
algorithms = ["adjacent", "orientation", "placement", "hunt_target", "monte_carlo"]

for offset, name in enumerate(algorithms, start=1):
    algo_rng = np.random.default_rng(298 + offset)
    result = ALGORITHMS[name](
        board, prior, algo_rng, game_config, search_config, mc_config
    )
    print(f"{name}: {result.shots} shots")


In [ ]:
benchmark_config = BenchmarkConfig(boards=50, seed=298)
prior_name, prior_fn = "uniform", uniform_prior

results = benchmark_algorithms(
    algorithms, prior_fn, benchmark_config, game_config, search_config, mc_config
)

print(f"Benchmark results (prior={prior_name})")
for name in algorithms:
    result = results[name]
    print(
        f"{name:12s} mean={result.mean:6.2f} std={result.std:6.2f} "
        f"median={result.median:6.2f} min={result.min_shots:3d} max={result.max_shots:3d}"
    )
